## Table of Contents

- [Prework](#Prework)
    - [Comparison between IV and HV](#Comparison_between_IV_and_HV)
    - [Visualisaton for IV and HV](#Visualisaton_for_IV_and_HV)
        - [Summary statistics](#Summary_statistics)
        - [Data visualization](#Data_visualization)
        - [Stationarity check](#Stationarity_check)
        - [Performance metrics](#Performance_metrics)

- [Build Model](#Build_Model)
    - [Split trainning and testing dataset using VIX](#Split_trainning_and_testing_dataset_using_VIX)
    - [Benchmark - Multi-Layer Perceptron (MLP)](#Benchmark_Multi-Layer_Perceptron_(MLP))
    - [Build the ARIMA model using IV](#Build_the_ARIMA_model_using_IV)
    - [Build the GARCH model using HV](#Build_the_GARCH_model_using_HV)
    - [Build ANN model - LSTM](#Build_ANN_model_LSTM)
    - [Build ANN model - GRU](#Build_ANN_model_GRU)
    - [Build LSTM-ARIMA model](#Build_LSTM-ARIMA_model)
    - [Build ARIMA-LSTM model](#Build_ARIMA-LSTM_model)
    - [Build GRU-ARIMA model](#Build_GRU-ARIMA_model)
    - [Build ARIMA-GRU model](#Build_ARIMA-GRU_model)

- [Strategies](#Strategies)
    - [Combine models prediction results](#Combine_models_prediction_results)
    - [Evaluation method 1: index metrics](#Evaluation_method_1_index_metrics)
    - [Evaluation method 2: trading strategies](#Evaluation_method_2_trading_strategies)
    - [Evaluation method 3: Option pricing](#Evaluation_method_3_Option_pricing)
        - [Call option](#Call_option)
        - [Put option](#Put_option)


## Prework

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
from scipy.optimize import minimize
from keras.layers.core import Dense, Activation, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.descriptivestats import describe
from statsmodels.tsa.stattools import adfuller as adf
import tensorflow as tf
import random as rn
import os
from scipy.stats import norm
import statsmodels.api as sm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import arch
import scipy.stats as stats
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.model_selection import GridSearchCV
from arch import arch_model
from statsmodels.tsa.arima.model import ARIMA
from pmdarima.arima import auto_arima
import math
from scipy.stats import norm
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
#Import data from downloaded excel files
VIX = pd.read_excel("VIX.xlsx",index_col = "Date")
SP500 = pd.read_excel("S&P500.xlsx",index_col = "Date")

### Comparison_between_IV_and_HV

In [ ]:
common_index = SP500.index.intersection(VIX.index)
vix_sub_df = VIX.loc[common_index]
# Calculate the historical volatility from the SP500 dataframe
window_size = 30  # Adjust the window size as desired
sp500_returns = SP500['Return'] # Calculate the daily returns
historical_volatility = sp500_returns.rolling(window_size).std()  # Calculate rolling standard deviation

# Normalize the data
historical_volatility_normalized = (historical_volatility - historical_volatility.mean()) / historical_volatility.std()
vix_normalized = (vix_sub_df['CBOE S&P500 Volatility Index - Close'] - vix_sub_df['CBOE S&P500 Volatility Index - Close'].mean()) / vix_sub_df['CBOE S&P500 Volatility Index - Close'].std()

# Plot the normalized historical volatility and VIX data
plt.figure(figsize=(12, 6))
plt.plot(historical_volatility_normalized, label='Historical Volatility')
plt.plot(vix_normalized, label='VIX')
plt.xlabel('Date')
plt.ylabel('Normalized Volatility')
plt.title('Normalized Historical Volatility vs VIX')
plt.legend()
plt.show()

### Visualisaton_for_IV_and_HV

#### Summary_statistics

In [ ]:
print(VIX['CBOE S&P500 Volatility Index - Close'].describe())

#### Data_visualization

In [ ]:
plt.figure(figsize=(12, 6))
# Create a line plot of the VIX close price
plt.plot(VIX.index, VIX['CBOE S&P500 Volatility Index - Close'])

# Set the title and axis labels
plt.title('CBOE S&P500 Volatility Index')
plt.xlabel('Date')
plt.ylabel('Close value')

# Display the plot
plt.show()

#### Stationarity_check

In [ ]:
VIX_close = VIX['CBOE S&P500 Volatility Index - Close'].dropna()

In [ ]:
from statsmodels.tsa.stattools import adfuller, kpss

# Plot rolling mean and standard deviation
def plot_rolling_statistics(timeseries, window=7):
    rolmean = timeseries.rolling(window=window).mean()
    rolstd = timeseries.rolling(window=window).std()
    
    plt.figure(figsize=(12, 6))
    plt.plot(timeseries, label='Original')
    plt.plot(rolmean, label='Rolling Mean')
    plt.plot(rolstd, label='Rolling Std')
    plt.legend()
    plt.title('Rolling Mean and Standard Deviation')
    plt.show()

# Augmented Dickey-Fuller test
def adf_test(timeseries):
    result = adfuller(timeseries, autolag='AIC')
    print("ADF Test Results:")
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    print(f'Critical Values:')
    for key, value in result[4].items():
        print(f'   {key}: {value}')

# Kwiatkowski-Phillips-Schmidt-Shin test
def kpss_test(timeseries):
    result = kpss(timeseries, regression='c')
    print("KPSS Test Results:")
    print(f'KPSS Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    print(f'Critical Values:')
    for key, value in result[3].items():
        print(f'   {key}: {value}')

# Check stationarity by plotting rolling statistics and conducting tests
plot_rolling_statistics(VIX['CBOE S&P500 Volatility Index - Close'])

adf_test(VIX_close)

kpss_test(VIX_close)


#### Performance_metrics

In [ ]:
# Calculate performance metrics
def evaluation(test_data, model_predictions):
    mse = mean_squared_error(test_data, model_predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(test_data, model_predictions)
    r2 = r2_score(test_data, model_predictions)

    print("Performance Metrics:")
    print("Mean Squared Error (MSE):", mse)
    print("Root Mean Squared Error (RMSE):", rmse)
    print("Mean Absolute Error (MAE):", mae)
    print("R-squared (R^2) Score:", r2)

## Build_Model

### Split_trainning_and_testing_dataset_using_VIX

In [ ]:
# Convert to the weekly dataset 
volatility_data = VIX['CBOE S&P500 Volatility Index - Close'].dropna().resample("W").last()

# Split the data into training, validation and test sets
train_size = int(len(volatility_data)*0.8)
test_size = len(volatility_data) - train_size

train_data = volatility_data[:train_size]
test_data = volatility_data[train_size:]
train_dates = train_data.index
test_dates = test_data.index

# Plot the data
plt.figure(figsize=(10,6))
plt.plot(train_data, label="Training data", color="blue")
plt.plot(test_data, label="Testing data", color="orange")
plt.title("CBOE S&P500 Volatility Index data split")
plt.xlabel("Time")
plt.ylabel("Volatility")
plt.legend()
plt.show()

### Benchmark_Multi-Layer_Perceptron_(MLP)

In [ ]:
# Normalize the data
scaler = MinMaxScaler()
train_data_normalized = scaler.fit_transform(train_data.values.reshape(-1, 1))

# Prepare the training data
lookback = 60  # Number of previous time steps to consider
X_train = []
y_train = []

for i in range(lookback, len(train_data_normalized)):
    X_train.append(train_data_normalized[i - lookback:i, 0])
    y_train.append(train_data_normalized[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape the input data for LSTM (batch_size, timesteps, features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]))

# Build the MLP model
model_mlp = Sequential()
model_mlp.add(Dense(units=64, activation='relu', input_shape=(X_train.shape[1],)))
model_mlp.add(Dense(units=32, activation='relu'))
model_mlp.add(Dense(units=1))
model_mlp.compile(optimizer='adam', loss='mean_squared_error')

# Train the MLP model
model_mlp.fit(X_train, y_train, epochs=10, batch_size=32)

# Prepare the test data
test_data_normalized = scaler.transform(test_data.values.reshape(-1, 1))

X_test = []
y_test = []

for i in range(lookback, len(test_data_normalized)):
    X_test.append(test_data_normalized[i - lookback:i, 0])
    y_test.append(test_data_normalized[i, 0])

X_test, y_test = np.array(X_test), np.array(y_test)

# Reshape the input data for MLP (batch_size, timesteps, features)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1]))

# Make predictions on the test data
predicted_values_mlp = model_mlp.predict(X_test)
predicted_values_mlp = scaler.inverse_transform(predicted_values_mlp)


In [ ]:
# Plot the graph
plt.figure(figsize=(12, 6))
plt.plot(test_dates[lookback:], test_data[lookback:], color='green', label='Test Data')
plt.plot(test_dates[lookback:], predicted_values_mlp, color='red', linestyle='dashed', label='Benchmark')
plt.title('Test Data and Predicted Values (MLP)')
plt.xlabel('Date')
plt.ylabel('VIX index')
plt.legend()
plt.show()

### Build_the_ARIMA_model_using_IV

Build a GARCH(1,1) model, and use the model to calculate fitted and predicted volatility of SP500 index. Plot the graph for original historical volatility and the predicted value, for the predicted value, use different color to show the fitted and predicted datasets.

In [ ]:
# Train AutoARIMA model
model = auto_arima(train_data, trace=True, error_action='ignore', suppress_warnings=True)
# Fit the model
model.fit(train_data)

In [ ]:
history = [x for x in train_data]
predicted_values_arima = []
N_test_observations = len(test_data)
for time_point in range(N_test_observations):
    model = ARIMA(history, order=(2,1,1))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    predicted_values_arima.append(yhat)
    true_test_value = test_data[time_point]
    history.append(true_test_value)


In [ ]:
# Plot the graph
plt.figure(figsize=(12, 6))

# Plot the test data with a solid green line
plt.plot(test_dates[60:], test_data[60:], color='green', label='Test Data', linestyle='-')

# Plot the predicted values from the ARIMA model with a dashed red line
plt.plot(test_dates[60:], predicted_values_arima[60:], color='red', linestyle='-', label='Predicted VIX Index (ARIMA model)')

# Plot the predicted values from the MLP model with a dash-dot blue line
plt.plot(test_dates[60:], predicted_values_mlp, color='blue', linestyle='--', label='Benchmark')

plt.title('Test Data and Predicted Values')
plt.xlabel('Date')
plt.ylabel('VIX index')

# Move the legend outside the plot area to avoid overlapping the lines
plt.legend()

plt.show()


In [ ]:
evaluation(test_data, predicted_values_arima)

### Build_the_GARCH_model_using_HV

In [ ]:
#Split train and test data using sp500 return data
sp500_data = sp500_returns.dropna().resample("W").last()
# Split the data into training, validation and test sets
train2_size = int(len(sp500_data)*0.8)
test2_size = len(sp500_data) - train2_size

train2_data = sp500_data[:train2_size]
test2_data = sp500_data[train2_size:]
train2_dates = train2_data.index
test2_dates = test2_data.index

# Plot the data
plt.figure(figsize=(10,6))
plt.plot(train2_data, label="Training data", color="blue")
plt.plot(test2_data, label="Testing data", color="orange")
plt.title("S&P500 return data split")
plt.xlabel("Time")
plt.ylabel("Return")
plt.legend()
plt.show()

In [ ]:
# Define the range of p and q values to consider
p_values = range(1, 3)  # Change the range as per your preference
q_values = range(1, 3)  # Change the range as per your preference

# Initialize lists to store AIC and BIC values
aic_values = []
bic_values = []
model_indices = []

# Iterate over different p and q values
for p in p_values:
    for q in q_values:
        model = arch.arch_model(sp500_data, vol='Garch', p=p, q=q)
        model_fit = model.fit(disp='off')
        aic_values.append(model_fit.aic)
        bic_values.append(model_fit.bic)
        model_indices.append((p, q))

# Plot AIC values
plt.plot(range(len(aic_values)), aic_values, marker='o')
plt.xlabel('Model Index')
plt.ylabel('AIC Value')
plt.title('AIC Values for GARCH Models')
plt.xticks(range(len(aic_values)), model_indices)
plt.show()

# Plot BIC values
plt.plot(range(len(bic_values)), bic_values, marker='o')
plt.xlabel('Model Index')
plt.ylabel('BIC Value')
plt.title('BIC Values for GARCH Models')
plt.xticks(range(len(bic_values)), model_indices)
plt.show()

In [ ]:
history = [x for x in train2_data]
predicted_values_garch = []
N_test_observations = len(test2_data)
for time_point in range(N_test_observations):
    model = arch.arch_model(history, vol='Garch', p=1, q=1)
    model_fit = model.fit()
    output = model_fit.forecast().variance.values[-1]
    yhat = np.sqrt(output[0])
    predicted_values_garch.append(yhat)
    true_test_value = test2_data[time_point]
    history.append(true_test_value)

In [ ]:
# Plot the test data and predicted volatility
plt.plot(test2_data.index, test2_data, color='blue', label='Test Data')
plt.plot(test2_data.index, predicted_values_garch, color='red', label='Predicted Volatility')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.title('Comparison of Test Data and Predicted Volatility')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
evaluation(test2_data, predicted_values_garch)

ARIMA model performance is better than GARCH model. The following hybrid models will use ARIMA model as the econometric model.

### Build_ANN_model_LSTM

In [ ]:
#Hyperparemeter tuning: grid search for hyperparameter tuning.

In [ ]:
# Normalize the data
scaler = MinMaxScaler()
train_data_normalized = scaler.fit_transform(train_data.values.reshape(-1, 1))
test_data_normalized = scaler.transform(test_data.values.reshape(-1, 1))

# Prepare the training data
lookback = 60  # Number of previous time steps to consider

def create_sequences(data, lookback):
    X, y = [], []
    for i in range(lookback, len(data)):
        X.append(data[i - lookback:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_data_normalized, lookback)
X_test, y_test = create_sequences(test_data_normalized, lookback)

# Reshape the input data for LSTM (batch_size, timesteps, features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Define the LSTM model
def create_lstm_model(units=50, learning_rate=0.001):
    model = Sequential()
    model.add(LSTM(units=units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(units=units))
    model.add(Dense(units=1))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Wrap the Keras model as a scikit-learn estimator
regressor = KerasRegressor(build_fn=create_lstm_model)

# Perform hyperparameter tuning using grid search
param_grid = {'units': [50, 100], 'batch_size': [32, 64], 'epochs': [10, 20], 'learning_rate': [0.001, 0.01, 0.1]}
model = GridSearchCV(estimator=regressor, param_grid=param_grid, cv=3)
model.fit(X_train, y_train)

# Get the best hyperparameters
best_units = model.best_params_['units']
best_batch_size = model.best_params_['batch_size']
best_epochs = model.best_params_['epochs']
best_learning_rate = model.best_params_['learning_rate']

print(f"Best Hyperparameters: units={best_units}, batch_size={best_batch_size}, epochs={best_epochs}, learning_rate={best_learning_rate}")

# Train the LSTM model with the best hyperparameters
final_model = create_lstm_model(units=best_units, learning_rate=best_learning_rate)
final_model.fit(X_train, y_train, epochs=best_epochs, batch_size=best_batch_size)

# Make predictions on the test data
predicted_values_lstm = final_model.predict(X_test)
predicted_values_lstm = scaler.inverse_transform(predicted_values_lstm)

# Plot the graph
plt.figure(figsize=(12, 6))
plt.plot(test_dates[lookback:], test_data[lookback:], color='green', label='Test Data')
plt.plot(test_dates[lookback:], predicted_values_lstm, color='red', linestyle='dashed', label='Predicted VIX Index (LSTM)')
plt.title('Test Data and Predicted Values (LSTM)')
plt.xlabel('Date')
plt.ylabel('VIX index')
plt.legend()
plt.show()


In [ ]:
#Original version
# Normalize the data
scaler = MinMaxScaler()
train_data_normalized = scaler.fit_transform(train_data.values.reshape(-1, 1))

# Prepare the training data
lookback = 60  # Number of previous time steps to consider
X_train = []
y_train = []

for i in range(lookback, len(train_data_normalized)):
    X_train.append(train_data_normalized[i - lookback:i, 0])
    y_train.append(train_data_normalized[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape the input data for LSTM (batch_size, timesteps, features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Build the LSTM model
model_lstm = Sequential()
model_lstm.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model_lstm.add(LSTM(units=50))
model_lstm.add(Dense(units=1))
model_lstm.compile(optimizer='adam', loss='mean_squared_error')

# Train the LSTM model
model_lstm.fit(X_train, y_train, epochs=10, batch_size=32)

# Prepare the test data
test_data_normalized = scaler.transform(test_data.values.reshape(-1, 1))

X_test = []
y_test = []

for i in range(lookback, len(test_data_normalized)):
    X_test.append(test_data_normalized[i - lookback:i, 0])
    y_test.append(test_data_normalized[i, 0])

X_test, y_test = np.array(X_test), np.array(y_test)

# Reshape the input data for LSTM (batch_size, timesteps, features)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Make predictions on the test data
predicted_values_lstm = model_lstm.predict(X_test)
predicted_values_lstm = scaler.inverse_transform(predicted_values_lstm)

In [ ]:
evaluation(test_data[lookback:], predicted_values_lstm)

In [ ]:
#Best Hyperparameters: units=50, batch_size=64, epochs=20, learning_rate=0.01

In [ ]:
#Optimised version
# Normalize the data
scaler = MinMaxScaler()
train_data_normalized = scaler.fit_transform(train_data.values.reshape(-1, 1))

# Prepare the training data
lookback = 60  # Number of previous time steps to consider
X_train = []
y_train = []

for i in range(lookback, len(train_data_normalized)):
    X_train.append(train_data_normalized[i - lookback:i, 0])
    y_train.append(train_data_normalized[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape the input data for LSTM (batch_size, timesteps, features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Build the LSTM model
model_lstm = Sequential()
model_lstm.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model_lstm.add(LSTM(units=50))
model_lstm.add(Dense(units=1))
optimizer = Adam(learning_rate=0.01)
model_lstm.compile(optimizer='adam', loss='mean_squared_error')

# Train the LSTM model
model_lstm.fit(X_train, y_train, epochs=20, batch_size=64)

# Prepare the test data
test_data_normalized = scaler.transform(test_data.values.reshape(-1, 1))

X_test = []
y_test = []

for i in range(lookback, len(test_data_normalized)):
    X_test.append(test_data_normalized[i - lookback:i, 0])
    y_test.append(test_data_normalized[i, 0])

X_test, y_test = np.array(X_test), np.array(y_test)

# Reshape the input data for LSTM (batch_size, timesteps, features)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Make predictions on the test data
predicted_values_lstm = model_lstm.predict(X_test)
predicted_values_lstm = scaler.inverse_transform(predicted_values_lstm)

In [ ]:
# Plot the graph
plt.figure(figsize=(12, 6))
plt.plot(test_dates[lookback:], test_data[lookback:], color='green', label='Test Data')
plt.plot(test_dates[lookback:], predicted_values_lstm, color='red', linestyle='-', label='Predicted VIX Index (LSTM)')
plt.plot(test_dates[lookback:], predicted_values_mlp, color='blue', linestyle='--', label='Benchmark')
plt.title('Test Data and Predicted Values (LSTM)')
plt.xlabel('Date')
plt.ylabel('VIX index')
plt.legend()
plt.show()

In [ ]:
evaluation(test_data[lookback:], predicted_values_lstm)

In [ ]:
evaluation(test_data[lookback:], predicted_values_mlp)

### Build_ANN_model_GRU

In [ ]:
#Original version
# Normalize the data
scaler = MinMaxScaler()
train_data_normalized = scaler.fit_transform(train_data.values.reshape(-1, 1))

# Prepare the training data
lookback = 60  # Number of previous time steps to consider
X_train = []
y_train = []

for i in range(lookback, len(train_data_normalized)):
    X_train.append(train_data_normalized[i - lookback:i, 0])
    y_train.append(train_data_normalized[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape the input data for GRU (batch_size, timesteps, features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Build the GRU model
model_gru = Sequential()
model_gru.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model_gru.add(GRU(units=50))
model_gru.add(Dense(units=1))
model_gru.compile(optimizer='adam', loss='mean_squared_error')

# Train the GRU model
model_gru.fit(X_train, y_train, epochs=10, batch_size=32)

# Prepare the test data
test_data_normalized = scaler.transform(test_data.values.reshape(-1, 1))

X_test = []
y_test = []

for i in range(lookback, len(test_data_normalized)):
    X_test.append(test_data_normalized[i - lookback:i, 0])
    y_test.append(test_data_normalized[i, 0])

X_test, y_test = np.array(X_test), np.array(y_test)

# Reshape the input data for GRU (batch_size, timesteps, features)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Make predictions on the test data
predicted_values_gru = model_gru.predict(X_test)
predicted_values_gru = scaler.inverse_transform(predicted_values_gru)


In [ ]:
evaluation(test_data[lookback:], predicted_values_gru)

In [ ]:
#Optimised version
# Normalize the data
scaler = MinMaxScaler()
train_data_normalized = scaler.fit_transform(train_data.values.reshape(-1, 1))

# Prepare the training data
lookback = 60  # Number of previous time steps to consider
X_train = []
y_train = []

for i in range(lookback, len(train_data_normalized)):
    X_train.append(train_data_normalized[i - lookback:i, 0])
    y_train.append(train_data_normalized[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape the input data for GRU (batch_size, timesteps, features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Build the GRU model
model_gru = Sequential()
model_gru.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model_gru.add(GRU(units=50))
model_gru.add(Dense(units=1))
optimizer = Adam(learning_rate=0.01)
model_gru.compile(optimizer='adam', loss='mean_squared_error')

# Train the GRU model
model_gru.fit(X_train, y_train, epochs=20, batch_size=64)

# Prepare the test data
test_data_normalized = scaler.transform(test_data.values.reshape(-1, 1))

X_test = []
y_test = []

for i in range(lookback, len(test_data_normalized)):
    X_test.append(test_data_normalized[i - lookback:i, 0])
    y_test.append(test_data_normalized[i, 0])

X_test, y_test = np.array(X_test), np.array(y_test)

# Reshape the input data for GRU (batch_size, timesteps, features)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Make predictions on the test data
predicted_values_gru = model_gru.predict(X_test)
predicted_values_gru = scaler.inverse_transform(predicted_values_gru)


In [ ]:
# Plot the graph
plt.figure(figsize=(12, 6))
plt.plot(test_dates[lookback:], test_data[lookback:], color='green', label='Test Data')
plt.plot(test_dates[lookback:], predicted_values_gru, color='red', linestyle='-', label='Predicted VIX Index (GRU)')
plt.plot(test_dates[lookback:], predicted_values_mlp, color='blue', linestyle='--', label='Benchmark')


plt.title('Test Data and Predicted Values (GRU)')
plt.xlabel('Date')
plt.ylabel('VIX index')
plt.legend()
plt.show()

In [ ]:
evaluation(test_data[lookback:], predicted_values_gru)

### Build_LSTM-ARIMA_model

In [ ]:
model1_predictions_series = pd.Series(predicted_values_arima, index=test_dates)
# Normalize the data
scaler = MinMaxScaler()
train_data_normalized = scaler.fit_transform(model1_predictions_series.values.reshape(-1, 1))

# Prepare the training data
lookback = 60  # Number of previous time steps to consider
X_train = []
y_train = []

for i in range(lookback, len(train_data_normalized)):
    X_train.append(train_data_normalized[i - lookback:i, 0])
    y_train.append(train_data_normalized[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape the input data for LSTM (batch_size, timesteps, features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Build the LSTM model
model_lstm = Sequential()
model_lstm.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model_lstm.add(LSTM(units=50))
model_lstm.add(Dense(units=1))
optimizer = Adam(learning_rate=0.01)
model_lstm.compile(optimizer='adam', loss='mean_squared_error')

# Train the LSTM model
model_lstm.fit(X_train, y_train, epochs=20, batch_size=64)

# Prepare the test data
test_data_normalized = scaler.transform(test_data.values.reshape(-1, 1))

X_test = []
y_test = []

for i in range(lookback, len(test_data_normalized)):
    X_test.append(test_data_normalized[i - lookback:i, 0])
    y_test.append(test_data_normalized[i, 0])

X_test, y_test = np.array(X_test), np.array(y_test)

# Reshape the input data for LSTM (batch_size, timesteps, features)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Make predictions on the test data
predicted_values_lstm_arima = scaler.inverse_transform(model_lstm.predict(X_test))

In [ ]:
# Plot the graph
plt.figure(figsize=(12, 6))
plt.plot(test_dates[lookback:], test_data[lookback:], color='green', label='Test Data')
plt.plot(test_dates[lookback:], predicted_values_lstm_arima, color='red', linestyle='-', label='Predicted VIX Index (LSTM-ARIMA)')
plt.plot(test_dates[lookback:], predicted_values_mlp, color='blue', linestyle='--', label='Benchmark')


plt.title('Test Data and Predicted Values (LSTM_ARIMA)')
plt.xlabel('Date')
plt.ylabel('VIX index')
plt.legend()
plt.show()


In [ ]:
evaluation(test_data[lookback:], predicted_values_lstm_arima)

### Build_ARIMA-LSTM_model

In [ ]:
# Train AutoARIMA model
history = [x[0] for x in predicted_values_lstm]
model = auto_arima(history, trace=True, error_action='ignore', suppress_warnings=True)
# Fit the model
model.fit(history)

In [ ]:
#One_step ahead forecast
predicted_values_arima_lstm = []
N_test_observations = len(test_data)
for time_point in range(N_test_observations):
    model = ARIMA(history, order=(3,1,3))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    predicted_values_arima_lstm.append(yhat)
    true_test_value = test_data[time_point]
    history.append(true_test_value)


In [ ]:
# Plot the graph
plt.figure(figsize=(12, 6))
plt.plot(test_dates[60:], test_data[60:], color='green', label='Test Data')
plt.plot(test_dates[60:], predicted_values_arima_lstm[60:], color='red', linestyle='-', label='Predicted VIX Index (ARIMA-LSTM)')
plt.plot(test_dates[60:], predicted_values_mlp, color='blue', linestyle='--', label='Benchmark')


plt.title('Test Data and Predicted Values (ARIMA-LSTM)')
plt.xlabel('Date')
plt.ylabel('VIX index')
plt.legend()
plt.show()

In [ ]:
evaluation(test_data, predicted_values_arima_lstm)

### Build_GRU-ARIMA_model

In [ ]:
model1_predictions_series = pd.Series(predicted_values_arima, index=test_dates)
# Normalize the data
scaler = MinMaxScaler()
train_data_normalized = scaler.fit_transform(model1_predictions_series.values.reshape(-1, 1))

# Prepare the training data
lookback = 60  # Number of previous time steps to consider
X_train = []
y_train = []

for i in range(lookback, len(train_data_normalized)):
    X_train.append(train_data_normalized[i - lookback:i, 0])
    y_train.append(train_data_normalized[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape the input data for GRU (batch_size, timesteps, features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Build the GRU model
model_gru = Sequential()
model_gru.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model_gru.add(GRU(units=50))
model_gru.add(Dense(units=1))
optimizer = Adam(learning_rate=0.01)
model_gru.compile(optimizer='adam', loss='mean_squared_error')

# Train the GRU model
model_gru.fit(X_train, y_train, epochs=20, batch_size=64)

# Prepare the test data
test_data_normalized = scaler.transform(test_data.values.reshape(-1, 1))

X_test = []
y_test = []

for i in range(lookback, len(test_data_normalized)):
    X_test.append(test_data_normalized[i - lookback:i, 0])
    y_test.append(test_data_normalized[i, 0])

X_test, y_test = np.array(X_test), np.array(y_test)

# Reshape the input data for GRU (batch_size, timesteps, features)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Make predictions on the test data
predicted_values_gru_arima = scaler.inverse_transform(model_gru.predict(X_test))

In [ ]:
# Plot the graph
plt.figure(figsize=(12, 6))
plt.plot(test_dates[lookback:], test_data[lookback:], color='green', label='Test Data')
plt.plot(test_dates[lookback:], predicted_values_gru_arima, color='red', linestyle='-', label='Predicted VIX Index (GRU-ARIMA)')
plt.plot(test_dates[lookback:], predicted_values_mlp, color='blue', linestyle='--', label='Benchmark')

plt.title('Test Data and Predicted Values (GRU-ARIMA)')
plt.xlabel('Date')
plt.ylabel('VIX index')
plt.legend()
#plt.grid(True)
plt.show()

### Build_ARIMA-GRU_model

In [ ]:
# Train AutoARIMA model
history = [x[0] for x in predicted_values_gru]
model = auto_arima(history, trace=True, error_action='ignore', suppress_warnings=True)
# Fit the model
model.fit(history)

In [ ]:
#One_step ahead forecast
predicted_values_arima_gru = []
N_test_observations = len(test_data)
for time_point in range(N_test_observations):
    model = ARIMA(history, order=(2,1,1))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    predicted_values_arima_gru.append(yhat)
    true_test_value = test_data[time_point]
    history.append(true_test_value)


In [ ]:
# Plot the graph
plt.figure(figsize=(12, 6))
plt.plot(test_dates[60:], test_data[60:], color='green', label='Test Data')
plt.plot(test_dates[60:], predicted_values_arima_gru[60:], color='red', linestyle='-', label='Predicted VIX Index (ARIMA-GRU)')
plt.plot(test_dates[lookback:], predicted_values_mlp, color='blue', linestyle='--', label='Benchmark')

plt.title('Test Data and Predicted Values (GARCH-GRU)')
plt.xlabel('Date')
plt.ylabel('VIX index')
plt.legend()
plt.show()

In [ ]:
evaluation(test_data, predicted_values_arima_gru)

## Strategies

### Combine_models_prediction_results

In [ ]:
y_lstm = pd.Series([x[0] for x in predicted_values_lstm], index = test_dates[60:])
y_lstm_arima = pd.Series([x[0] for x in predicted_values_lstm_arima], index = test_dates[60:])
y_arima_lstm = pd.Series(predicted_values_arima_lstm[60:],index =test_dates[60:])
y_gru = pd.Series([x[0] for x in predicted_values_gru], index = test_dates[60:])
y_gru_arima = pd.Series([x[0] for x in predicted_values_gru_arima], index = test_dates[60:])
y_arima_gru = pd.Series(predicted_values_arima_gru[60:],index =test_dates[60:])
y_benchmark = pd.Series([x[0] for x in predicted_values_mlp], index = test_dates[60:])
y_test = test_data[60:]

In [ ]:
plt.figure(figsize=(12, 6))
# Plot the y-values
plt.plot(y_lstm, label='LSTM')
plt.plot(y_lstm_arima, label='LSTM-ARIMA')
plt.plot(y_arima_lstm, label='ARIMA-LSTM')
plt.plot(y_gru, label='GRU')
plt.plot(y_gru_arima, label='GRU-ARIMA')
plt.plot(y_arima_gru, label='ARIMA-GRU')
plt.plot(y_benchmark, label='Benchmark')
plt.plot(y_test, label='Test Data')

# Set the title and labels
plt.title('Comparison of Predicted Values')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.legend()

# Display the plot
plt.show()

In [ ]:
models = ["Benchmark",'LSTM', 'LSTM_ARIMA', 'ARIMA_LSTM', 'GRU', 'GRU_ARIMA', 'ARIMA_GRU',"Test_data"]
predictions = [y_benchmark, y_lstm, y_lstm_arima, y_arima_lstm, y_gru, y_gru_arima, y_arima_gru,y_test]

data_vol = {model: prediction for model, prediction in zip(models, predictions)}

# Create the DataFrame
df_vol = pd.DataFrame(data_vol)

In [ ]:
df_vol

### Evaluation_method_1_index_metrics

In [ ]:
#Evaluation method 1: index metrics (Prediction accuracy) Ref: Main1
#1 Testing accuracy ratio for modelling (bar plot/box plot)
#2 MSE, RMSE, MAE, etc...

In [ ]:
# Calculate statistical measures for each model
metrics = ['MSE', 'RMSE', 'MAE', 'Correlation', 'Cosine Similarity']
results = []

for column in df_vol.columns[:-1]:  # Exclude the 'Test_data' column
    model_data = df_vol[column]
    # Calculate statistical measures
    mse = np.mean((model_data - df_vol['Test_data']) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(model_data - df_vol['Test_data']))
    correlation = np.corrcoef(model_data, df_vol['Test_data'])[0, 1]
    cosine_similarity = np.dot(model_data, df_vol['Test_data']) / (np.linalg.norm(model_data) * np.linalg.norm(df_vol['Test_data']))
    
    # Store the results
    results.append([mse, rmse, mae, correlation, cosine_similarity])

# Create a DataFrame to display the results
df_results = pd.DataFrame(results, columns=metrics, index=df_vol.columns[:-1])

# Sort the models based on the similarity metrics (choose the desired metric as per your preference)
df_results_sorted_vol = df_results.sort_values(by='MSE', ascending=True)

In [ ]:
df_results_sorted_vol

In [ ]:
df_vol

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Calculate relative MSE for each model
models = df_vol.columns[:-1]  # Extract the model names from df_vol
relative_mse_values = []

for model in models:
    mse_values = (df_vol['Test_data'] - df_vol[model]) ** 2
    relative_mse = mse_values / np.mean(mse_values)
    relative_mse_values.append(relative_mse)

# Create boxplots of relative MSE for different models
plt.figure(figsize=(10, 6))
plt.boxplot(relative_mse_values, vert=True, showfliers=False, notch=True, labels=models)
plt.xlabel('Models')
plt.ylabel('Relative MSE')
plt.title('Boxplot of Cross-Sectional Relative MSE for Different Models')
plt.show()


### Evaluation_method_2_trading_strategies

In [ ]:
#Input s&p500 historical data and create benchmark
sp500 = pd.read_csv("SPY.csv")
sp500.index = pd.to_datetime(sp500["Date"])
sp500_return= sp500.loc[y_test.index[0]:y_test.index[-1]]["Adj Close"].pct_change()
sp500_value = []
value = 1
for i in sp500_return[1:]:
    value *= (1+i)
    sp500_value.append(value)
sp500_value.insert(0,1)

In [ ]:
models = ["Benchmark",'LSTM', 'LSTM_ARIMA', 'ARIMA_LSTM', 'GRU', 'GRU_ARIMA', 'ARIMA_GRU']
predictions = [y_benchmark, y_lstm, y_lstm_arima, y_arima_lstm, y_gru, y_gru_arima, y_arima_gru]

In [ ]:
# Initialize portfolio values
portfolio_values = pd.DataFrame(index=y_test.index, columns=models)
y_test_return = y_test.pct_change()
# Calculate portfolio values for each model
for i, model in enumerate(models):
    signal_models = np.sign(predictions[i].diff())  # Calculate signal based on predicted values
    signal_test = np.sign(y_test.diff())
    signal = [-1 if signal_models[i] == signal_test[i] else 1 for i in range(len(signal_models))]
    portfolio_values[model] = (1 + signal * abs(y_test_return.shift(-3))).cumprod()  # Calculate portfolio values

portfolio_values["S&P500"] = sp500_value
# Plot portfolio values
plt.figure(figsize=(12, 6))
for model in models:
    plt.plot(portfolio_values.index, portfolio_values[model], label=model)
plt.plot(portfolio_values.index, sp500_value, label="S&P 500")
plt.title("Portfolio Values")
plt.xlabel("Date")
plt.ylabel("Value")
plt.legend()
plt.show()

In [ ]:
# Calculate daily returns for each model
daily_returns = portfolio_values.pct_change()

# Calculate Sharpe ratio
risk_free_rate = 0.0  # Set the risk-free rate
sharpe_ratio = (daily_returns.mean() - risk_free_rate) / daily_returns.std()

# Create table for daily returns
daily_returns_table = pd.DataFrame(daily_returns.mean(), columns=["Weekly Return"])
daily_returns_table["Std Dev"] = daily_returns.std()
daily_returns_table["Sharpe Ratio"] = sharpe_ratio
daily_returns_table = daily_returns_table.round(4)

# Sort the table by "Weekly Return" column in descending order
daily_returns_table = daily_returns_table.sort_values(by="Weekly Return", ascending=False)

# Display table for daily returns and Sharpe ratio
print("Weekly Returns and Sharpe Ratio:")
daily_returns_table

### Evaluation_method_3_Option_pricing

In [ ]:
#Black-Scholes model

In [ ]:
def black_scholes(S, K, r, T, sigma, option_type):
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    
    if option_type == 'call':
        price = S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        price = K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type. Choose 'call' or 'put'.")
    
    return price

#### Call_option

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Time series data for S&P 500 stock prices
sp500_price = sp500.loc[y_test.index[0]:y_test.index[-1]]["Adj Close"]

# Volatility predictions for each model
models = ["Benchmark", 'LSTM', 'LSTM_ARIMA', 'ARIMA_LSTM', 'GRU', 'GRU_ARIMA', 'ARIMA_GRU',"Test data"]
predictions = [y_benchmark, y_lstm, y_lstm_arima, y_arima_lstm, y_gru, y_gru_arima, y_arima_gru,y_test]

# Parameters for option pricing
risk_free_rate = 0.05  # Risk-free interest rate (r)
time_to_maturity = 0.5  # Time to option expiration in years (T)

# Generate option prices for each model
option_prices = []
for model, volatility in zip(models, predictions):
    option_prices_model = []
    for stock_price, sigma in zip(sp500_price, volatility):
        strike_price = stock_price + 50  # Strike price is $50 higher than S&P 500 price
        option_price = black_scholes(stock_price, strike_price, risk_free_rate, time_to_maturity, 0.01*sigma, 'call')
        option_prices_model.append(option_price)
    option_prices.append(option_prices_model)

# Create a DataFrame from the option prices
data_option = list(map(list, zip(*option_prices)))
df_option = pd.DataFrame(data_option, columns=models)
df_option = df_option.set_index(y_test.index)

In [ ]:
# Plotting the time series data for each model
plt.figure(figsize=(12, 6))
for column in df_option.columns:
    plt.plot(df_option.index, df_option[column], label=column)

# Set the title, x-axis label, and y-axis label
plt.title("Comparison of different models' predicted call option prices")
plt.xlabel('Date')
plt.ylabel('Call option price')

# Add a legend
plt.legend()

# Display the plot
plt.show()


In [ ]:
import numpy as np

# Calculate statistical measures for each model
metrics = ['MSE', 'RMSE', 'MAE', 'Correlation', 'Cosine Similarity']
results = []

for column in df_option.columns[:-1]:  # Exclude the 'Test_data' column
    model_data = df_option[column]
    
    # Calculate statistical measures
    mse = np.mean((model_data - df_option['Test data']) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(model_data - df_option['Test data']))
    correlation = np.corrcoef(model_data, df_option['Test data'])[0, 1]
    cosine_similarity = np.dot(model_data, df_option['Test data']) / (np.linalg.norm(model_data) * np.linalg.norm(df_option['Test data']))
    
    # Store the results
    results.append([mse, rmse, mae, correlation, cosine_similarity])

# Create a DataFrame to display the results
df_results = pd.DataFrame(results, columns=metrics, index=df_option.columns[:-1])

# Sort the models based on the similarity metrics (choose the desired metric as per your preference)
df_results_sorted_option = df_results.sort_values(by='MSE', ascending=True)

In [ ]:
df_results_sorted_option

#### Put_option

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Time series data for S&P 500 stock prices
sp500_price = sp500.loc[y_test.index[0]:y_test.index[-1]]["Adj Close"]

# Volatility predictions for each model
models = ["Benchmark", 'LSTM', 'LSTM_ARIMA', 'ARIMA_LSTM', 'GRU', 'GRU_ARIMA', 'ARIMA_GRU',"Test data"]
predictions = [y_benchmark, y_lstm, y_lstm_arima, y_arima_lstm, y_gru, y_gru_arima, y_arima_gru,y_test]

# Parameters for option pricing
risk_free_rate = 0.05  # Risk-free interest rate (r)
time_to_maturity = 0.5  # Time to option expiration in years (T)

# Generate option prices for each model
option_prices = []
for model, volatility in zip(models, predictions):
    option_prices_model = []
    for stock_price, sigma in zip(sp500_price, volatility):
        strike_price = stock_price - 50  # Strike price is $50 higher than S&P 500 price
        option_price = black_scholes(stock_price, strike_price, risk_free_rate, time_to_maturity, 0.01*sigma, 'put')
        option_prices_model.append(option_price)
    option_prices.append(option_prices_model)

# Create a DataFrame from the option prices
data_option = list(map(list, zip(*option_prices)))
df_option = pd.DataFrame(data_option, columns=models)
df_option = df_option.set_index(y_test.index)

In [ ]:
# Plotting the time series data for each model
plt.figure(figsize=(12, 6))
for column in df_option.columns:
    plt.plot(df_option.index, df_option[column], label=column)

# Set the title, x-axis label, and y-axis label
plt.title("Comparison of different models' predicted put option prices")
plt.xlabel('Date')
plt.ylabel('Put option price')

# Add a legend
plt.legend()

# Display the plot
plt.show()


In [ ]:
import numpy as np

# Calculate statistical measures for each model
metrics = ['MSE', 'RMSE', 'MAE', 'Correlation', 'Cosine Similarity']
results = []

for column in df_option.columns[:-1]:  # Exclude the 'Test_data' column
    model_data = df_option[column]
    
    # Calculate statistical measures
    mse = np.mean((model_data - df_option['Test data']) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(model_data - df_option['Test data']))
    correlation = np.corrcoef(model_data, df_option['Test data'])[0, 1]
    cosine_similarity = np.dot(model_data, df_option['Test data']) / (np.linalg.norm(model_data) * np.linalg.norm(df_option['Test data']))
    
    # Store the results
    results.append([mse, rmse, mae, correlation, cosine_similarity])

# Create a DataFrame to display the results
df_results = pd.DataFrame(results, columns=metrics, index=df_option.columns[:-1])

# Sort the models based on the similarity metrics (choose the desired metric as per your preference)
df_results_sorted_option = df_results.sort_values(by='MSE', ascending=True)

In [ ]:
df_results_sorted_option